<a href="https://colab.research.google.com/github/MayaPikachu/Offense-detector/blob/main/Offense_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install transformers
!pip install unidecode
!pip install tokenizers


In [8]:
!gdown --fuzzy "https://drive.google.com/uc?id=18VOWo-rHUiQ7pIlzzMzkFCrZNX0RIWCe"
!gdown --fuzzy "https://drive.google.com/uc?id=1G-bAySkxcI1laV9wDkWGF1mxENICzglN"


Downloading...
From: https://drive.google.com/uc?id=18VOWo-rHUiQ7pIlzzMzkFCrZNX0RIWCe
To: /content/SBIC.v2.trn.csv
100% 27.7M/27.7M [00:00<00:00, 83.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1G-bAySkxcI1laV9wDkWGF1mxENICzglN
To: /content/SBIC.v2.tst.csv
100% 4.40M/4.40M [00:00<00:00, 29.9MB/s]


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from transformers import AutoTokenizer
from unidecode import unidecode
from tokenizers import ByteLevelBPETokenizer

In [10]:
df = pd.read_csv("SBIC.v2.trn.csv")
dt = pd.read_csv("SBIC.v2.tst.csv")

In [11]:
poststrn = df.post.values
resultstrn = df.offensiveYN.values

poststst = dt.post.values
resultstst = dt.offensiveYN.values

tokenizer = ByteLevelBPETokenizer()
tokenizer.train_from_iterator([poststrn[:int(4E7)]], vocab_size=500, min_frequency=5)

vocab_size = tokenizer.get_vocab_size()

print(f"\n\n\tTokenizado! Vocab_size = {vocab_size}\n")

encode = lambda s: tokenizer.encode(s).ids
decode = lambda ids: tokenizer.decode(ids)



	Tokenizado! Vocab_size = 500



In [16]:
block_size = 30
batch_size = 64
iter = 3000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def get_data(file, results, batch_size = batch_size):
  X, Y = [], []
  for i in range(batch_size):
    ix = torch.randint(0, len(file), (1, ))
    input = (encode(file[ix]) + [0] * block_size)[:block_size]
    X.append(input)
    Y.append(results[ix])
  return torch.tensor(X).float(), torch.nan_to_num(torch.tensor(Y).float(), nan=0.0)

get_data(poststrn, resultstrn)

(tensor([[379., 326., 280.,  ..., 277., 271., 320.],
         [ 43.,  64.,  69.,  ..., 342.,  82., 311.],
         [ 41., 373., 394.,  ...,  85., 283., 285.],
         ...,
         [ 31.,  54., 280.,  ...,   0.,   0.,   0.],
         [ 49.,  64.,  66.,  ...,   0.,   0.,   0.],
         [ 51., 258., 262.,  ..., 266.,  82., 256.]]),
 tensor([1.0000, 0.0000, 0.5000, 0.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.5000, 1.0000, 0.0000,
         1.0000, 0.5000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000,
         1.0000, 1.0000, 0.5000, 0.0000, 0.0000, 1.0000, 0.5000, 0.0000, 0.0000,
         1.0000, 0.0000, 1.0000, 0.5000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000,
         1.0000, 0.0000, 0.0000, 0.5000, 0.5000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000]))

In [17]:
class MLP(nn.Module):
  def __init__(self, n_hidden):
    super().__init__()
    self.l1 = nn.Linear(block_size, n_hidden)
    self.l2 = nn.Linear(n_hidden, n_hidden)
    self.l3 = nn.Linear(n_hidden, 1)
    self.s = nn.Sigmoid()

  def forward(self, x):
    x = F.relu(self.l1(x))
    x = nn.Dropout(0.2)(F.relu(self.l2(x)))
    x = nn.Dropout(0.2)(self.l3(x))
    return self.s(x)



In [18]:
model = MLP(128)
model.to(device)
params = list(model.parameters())
optimizer = torch.optim.Adam(params, lr=0.0003)
loss_fn = nn.BCELoss()
for i in range(iter):
  X, Y = get_data(poststrn, resultstrn)
  X = X.to(device)
  Y = Y.to(device)
  optimizer.zero_grad()
  logits = model(X)
  loss = loss_fn(logits, Y.unsqueeze(1))
  loss.backward()
  optimizer.step()
  if (i%100 == 0):
    print(f'step = {i}, loss = {loss}')


step = 0, loss = 41.52538299560547
step = 100, loss = 2.025615930557251
step = 200, loss = 1.2374787330627441
step = 300, loss = 1.0146201848983765
step = 400, loss = 0.6715511083602905
step = 500, loss = 0.6472303867340088
step = 600, loss = 0.6880075931549072
step = 700, loss = 0.7195512056350708
step = 800, loss = 0.6726694107055664
step = 900, loss = 0.7258880138397217
step = 1000, loss = 0.657617449760437
step = 1100, loss = 0.7024713158607483
step = 1200, loss = 0.7306045293807983
step = 1300, loss = 0.6448284983634949
step = 1400, loss = 0.6326622366905212
step = 1500, loss = 0.6957944631576538
step = 1600, loss = 0.7472766041755676
step = 1700, loss = 0.6569730043411255
step = 1800, loss = 0.6665205955505371
step = 1900, loss = 0.674483060836792
step = 2000, loss = 0.6588184237480164
step = 2100, loss = 0.66803377866745
step = 2200, loss = 0.7171688675880432
step = 2300, loss = 0.6251607537269592
step = 2400, loss = 0.6151273250579834
step = 2500, loss = 0.6658488512039185
step

In [19]:
precisao = 0.0
for i in range(30):
  X, Y = get_data(poststst, resultstst)
  X = X.to(device)
  Y = Y.to(device)
  logits = model(X)
  precisao += (logits.round() == Y.unsqueeze(1)).float().mean()/30.
  print(f'x: {decode(X[1].int().to("cpu").tolist())}, y = {Y[1].to("cpu")}, predito = {0 if logits[1].to("cpu").item() < .5 else 1}')

print(precisao)


x: What do you call a Jewish helicopter? A holocaupter!!!!!!, y = 1.0, predito = 1
x: What’s the the difference between Al Qaeda base and a Pakist, y = 1.0, predito = 1
x: I will never in my life step foot into an Arab state other than, y = 1.0, predito = 1
x: Why does Japan have a low obesity rate and a low birth rate, y = 1.0, predito = 1
x: What do you call a Jew with autism? Auschwistic!!!!!!!!!!, y = 1.0, predito = 1
x: The fucking #fakenews @ABC, @NBCNews, @CBSNews, y = 1.0, predito = 0
x: 15 minutes. Nervous. Not because of speaking, but because of the, y = 0.0, predito = 0
x: Yeah, child labor IS illegal. Unless you mean "w, y = 1.0, predito = 1
x: whats the difference between a black person and a jew? Depends on the, y = 1.0, predito = 1
x: How do you pick up black girls? You burn their village, chain, y = 1.0, predito = 0
x: RT @Smith_Wessern: Some hoes live jus to hit the clubs, y = 1.0, predito = 0
x: How you gonna call my ex worthless when you fucked him too what the fuck!

In [20]:
sorted(tokenizer.get_vocab(), key=lambda x: tokenizer.get_vocab()[x])

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '|',
 '}',
 '~',
 '¡',
 '¢',
 '£',
 '¤',
 '¥',
 '¦',
 '§',
 '¨',
 '©',
 'ª',
 '«',
 '¬',
 '®',
 '¯',
 '°',
 '±',
 '²',
 '³',
 '´',
 'µ',
 '¶',
 '·',
 '¸',
 '¹',
 'º',
 '»',
 '¼',
 '½',
 '¾',
 '¿',
 'À',
 'Á',
 'Â',
 'Ã',
 'Ä',
 'Å',
 'Æ',
 'Ç',
 'È',
 'É',
 'Ê',
 'Ë',
 'Ì',
 'Í',
 'Î',
 'Ï',
 'Ð',
 'Ñ',
 'Ò',
 'Ó',
 'Ô',
 'Õ',
 'Ö',
 '×',
 'Ø',
 'Ù',
 'Ú',
 'Û',
 'Ü',
 'Ý',
 'Þ',
 'ß',
 'à',
 'á',
 'â',
 'ã',
 'ä',
 'å',
 'æ',
 'ç',
 'è',
 'é',
 'ê